In [2]:
#import python libraries
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler, normalize

In [3]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from dataframe_cleaner import DataFrameCleaner
from dataframe_outlier import *
from plot_data import *
from dataframe_info import DataFrameInfo
from user_overview import UserOverviewScript